In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import AveragePooling1D, Activation, Flatten, Dense
from keras.layers import GlobalMaxPooling1D, Dropout
from keras.layers import GlobalAveragePooling1D, UpSampling1D
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import SGD, Adam, Nadam, Adamax
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from data_in_2s import data_in, normali
from ori_model import ori_model
#from tqdm import tqdm
#from sklearn.model_selection import train_test_split

/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_test = 6
X_train, y_train = data_in(num_test)

In [3]:
#Nomalization
#y_train = (y_train-50)/150
#X_train = normali(X_train)
print('Shape of X_train is: ' + str(X_train.shape))
print('Shape of y_train is: ' + str(y_train.shape))

Shape of X_train is: (3225, 1000, 5)
Shape of y_train is: (3225, 1)


In [4]:
#Import and Split test data

name_x = 'Training_data/DATA_%.2d_TYPE02.mat'%num_test
in_tem_x = sio.loadmat(name_x)['sig']
in_tem_x = in_tem_x.T

name_y = 'Training_data/DATA_%.2d_TYPE02_BPMtrace.mat'%num_test
in_tem_y = sio.loadmat(name_y)['BPM0']

X_test_tem = np.zeros((in_tem_y.shape[0]*2, 125*8, 5))

k = 0

for j in range(in_tem_y.shape[0]*2-1):
	X_test_tem[j, :, :] =  in_tem_x[k:(k+125*8), 1:]
	k += 125

tem_y = np.zeros((in_tem_y.shape[0]*2, 1))
index = [a*2 for a in range(in_tem_y.shape[0])]
tem_y[index] = in_tem_y

for b in range(tem_y.shape[0]-1):
	if tem_y[b] == 0:
		tem_y[b] = (tem_y[b-1] + tem_y[b+1])/2.

y_test = tem_y[:-1, :]
X_test = X_test_tem[:-1, :, :]

#y_test = (y_test-50)/150
#X_test = normali(X_test)

print('The shape of X_test is: ' + str(X_test.shape))
print('The shape of y_test is: ' + str(y_test.shape))

The shape of X_test is: (299, 1000, 5)
The shape of y_test is: (299, 1)


In [5]:
#Convert to spectrum
#68 - 178

spe_len = 110
y_train = np.rint(y_train)
y_test = np.rint(y_test)

y_train_onehot = np.zeros((y_train.shape[0], spe_len))
y_test_onehot = np.zeros((y_test.shape[0], spe_len))

y_train = y_train - 68
y_test = y_test - 68

for i in range(y_train.shape[0]):
    y_train_onehot[i, int(y_train[i][0])] = 1
    
for i in range(y_test.shape[0]):
    y_test_onehot[i, int(y_test[i][0])] = 1
    
print('The shape of y_train_onehot is: ' + str(y_train_onehot.shape))
print('The shape of y_test_onehot is: ' + str(y_test_onehot.shape))


The shape of y_train_onehot is: (3225, 110)
The shape of y_test_onehot is: (299, 110)


In [6]:
model = ori_model((1000, 5), 8)

/Users/xufanxiong/Desktop/pytorch/ori_model.py:96: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model = Model(input=inputs, output=output)


In [7]:
optm = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=3*10**(-4))
model.compile(loss= "categorical_crossentropy", 
              optimizer=optm, 
              metrics=["accuracy"])

In [9]:
filepath="weight_spe.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks_list = [checkpoint]
history = model.fit(X_train, y_train_onehot, 
                    epochs=100, 
                    batch_size=128, 
                    validation_split=0.2, 
                    callbacks=callbacks_list,
                    shuffle=True,
                    verbose=1)

Train on 2580 samples, validate on 645 samples
Epoch 1/100
2580/2580 [==============================] - 77s 30ms/step - loss: 5.0598 - acc: 0.0233 - val_loss: 6.5718 - val_acc: 0.0109

Epoch 00001: val_acc improved from -inf to 0.01085, saving model to weight_troika_ori_2s.hdf5
Epoch 2/100
2580/2580 [==============================] - 71s 27ms/step - loss: 4.4252 - acc: 0.0539 - val_loss: 6.0910 - val_acc: 0.0124

Epoch 00002: val_acc improved from 0.01085 to 0.01240, saving model to weight_troika_ori_2s.hdf5
Epoch 3/100
2580/2580 [==============================] - 74s 29ms/step - loss: 3.9419 - acc: 0.1283 - val_loss: 7.1590 - val_acc: 0.0496

Epoch 00003: val_acc improved from 0.01240 to 0.04961, saving model to weight_troika_ori_2s.hdf5
Epoch 4/100
2580/2580 [==============================] - 77s 30ms/step - loss: 3.2865 - acc: 0.2562 - val_loss: 9.2718 - val_acc: 0.0295

Epoch 00004: val_acc did not improve from 0.04961
Epoch 5/100
2580/2580 [==============================] - 71s 28

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
filepath="weight_spe.hdf5"
model.load_weights(filepath)

In [ ]:
predict = model.predict(X_test)

In [ ]:
#evaluate
eva = model.evaluate(X_test, y_test_onehot, batch_size=32, verbose=1, sample_weight=None)
print()
print("Loss = " + str(eva[0]))
print("Test Accuracy = " + str(eva[1]))